In [1]:
from numpy import random
import numpy as np
import pandas as pd

# 不用dataframe，只用ndarray
# 计算速度：ndarray > Series > list > DataFrame

pd.reset_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
feature = pd.read_excel(r'./input/feature.xls')
# col = feature.columns[10:-1]
col = ['Leq_mean', 'Leq_std', 'Leq_25', 'Leq_median',
       'Leq_75', 'Leq_10-Leq_90', 'Loudness_mean', 'Loudness_std',
       'Loudness_25', 'Loudness_median', 'Loudness_75',
       'Loudness_10-Loudness_90', 'Roughness_mean', 'Roughness_std',
       'Roughness_25', 'Roughness_median', 'Roughness_75',
       'Roughness_10-Roughness_90', 'Sharpness_mean', 'Sharpness_std',
       'Sharpness_25', 'Sharpness_median', 'Sharpness_75',
       'Sharpness_10-Sharpness_90', 'Fluct_mean', 'Fluct_std', 'Fluct_25',
       'Fluct_median', 'Fluct_75', 'Fluct_10-Fluct_90', 'Tonality_mean',
       'Tonality_std', 'Tonality_25', 'Tonality_median', 'Tonality_75',
       'Tonality_10-Tonality_90', 'leq_w', 'leq_esm', 'sharpness_slop2', 'tonality_esm',
       'tonality_phimax', 'crz']
feature_use = feature[col]
k = 4

In [2]:
# 生成一个随机矩阵
a = random.randint(1, 100, size=(3, 4))


In [3]:
feature_use = feature_use.to_numpy()

In [4]:
print(type(feature_use))
feature_use

<class 'numpy.ndarray'>


array([[4.9880e+01, 1.3900e+00, 4.8840e+01, ..., 7.5760e-01, 1.2229e-01,
        2.1319e-01],
       [4.9710e+01, 1.6400e+00, 4.8370e+01, ..., 7.6335e-01, 1.1179e-01,
        2.2335e-01],
       [5.0530e+01, 3.0600e+00, 4.8450e+01, ..., 6.8687e-01, 8.9390e-02,
        4.2840e-02],
       ...,
       [2.0240e+01, 3.5500e+00, 1.7020e+01, ..., 7.5057e-01, 1.2466e-01,
        4.4000e-02],
       [1.8030e+01, 2.0400e+00, 1.6610e+01, ..., 7.5057e-01, 1.2466e-01,
        8.4090e-02],
       [2.9180e+01, 5.1500e+00, 2.6790e+01, ..., 7.5057e-01, 1.2466e-01,
        4.8283e-01]])

In [5]:
feature_use.shape[0], feature_use.shape[1]

(6747, 42)

In [6]:
# # 特征总数，即列数
# feature_num = feature_use.shape[1]
# # 声音片段总数，即行数
# voice_num = feature_use.shape[0]

In [7]:
a = np.array([1, 2, 3])
b = np.array([3, 4, 7])
(a - b) ** 2

array([ 4,  4, 16], dtype=int32)

In [8]:
# 计算两点的欧氏距离
def Euclidean_distance(x, y):
    distance = np.sqrt(np.sum((x - y) ** 2))
    return distance

In [9]:
# 聚类取平均值，用于计算更新聚类点
def cluster_mean(cluster):
    return np.mean(cluster, axis=0)

In [10]:
# 找到初始聚类点
def init_centroid(dataset):
    # 特征总数，即列数
    feature_num = dataset.shape[1]
    # 声音片段总数，即行数
    voice_num = dataset.shape[0]

    # 第一个聚类点随机选取
    # 之后每个聚类点取到之前距离最大的点
    centroid = np.zeros((4, feature_num))

    index = int(np.random.uniform(0, voice_num))
    centroid[0, :] = dataset[index, :]
    centroid_0 = centroid[0, :]

    centroid_1 = centroid_0
    centroid_2 = centroid_0
    centroid_3 = centroid_0
    max_distance = 0

    for i in range(voice_num):
        dataset_i = dataset[i, :]
        distance = Euclidean_distance(centroid_0, dataset_i)
        if distance > max_distance:
            centroid_1 = dataset_i
            max_distance = distance

    centroid[1, :] = centroid_1
    max_distance = 0

    for i in range(voice_num):
        dataset_i = dataset[i, :]
        distance = Euclidean_distance(centroid_0, dataset_i) + Euclidean_distance(centroid_1, dataset_i)
        if distance > max_distance:
            centroid_2 = dataset_i
            max_distance = distance

    centroid[2, :] = centroid_2
    max_distance = 0

    for i in range(voice_num):
        dataset_i = dataset[i, :]
        distance = Euclidean_distance(centroid_0, dataset_i) + Euclidean_distance(centroid_1,
                                                                                  dataset_i) + Euclidean_distance(
            centroid_2, dataset_i)
        if distance > max_distance:
            centroid_3 = dataset_i
            max_distance = distance

    centroid[3, :] = centroid_3

    return centroid

In [11]:
def Kmeans(dataset):
    # 特征总数，即列数
    feature_num = dataset.shape[1]
    # 声音片段总数，即行数
    voice_num = dataset.shape[0]

    # 初始化一个矩阵来储存每个点的簇分配结果
    # clusterAssment包含两列：1列记录簇索引值，2列存储当前点到簇质心的距离（用来评价聚类的效果）
    # 全部置为-1，因为有0类点
    cluster_assessment = np.ones((voice_num, 2))
    cluster_assessment = -cluster_assessment
    cluster_change = True

    centroid = init_centroid(dataset)

    # 初始化标志变量，用于判断迭代是否继续，如果True，则迭代继续
    while cluster_change:
        cluster_change = False

        # 遍历所有行
        for i in range(voice_num):
            min_distance = 10000
            # 距离最近的聚类点的索引（0,1,2,3)
            min_index = -1

            # 遍历寻找距离每个点最近的质心
            for j in range(k):
                distance = Euclidean_distance(centroid[j, :], dataset[i, :])
                # 如果距离小于minDist，更新minDist和index索引值
                if distance < min_distance:
                    min_distance = distance
                    min_index = j

            # 如果有任意一点的簇分配结果改变，则更新cluster_change为True
            if cluster_assessment[i, 0] != min_index:
                cluster_change = True
                cluster_assessment[i, :] = min_index, min_distance

        # 遍历所有质心，更新取值
        for i in range(k):
            # 获取相同簇质心的所有点，取均值更新簇质心
            same_cluster_index = (cluster_assessment[:, 0] == i)
            points_same_cluster = dataset[same_cluster_index]
            centroid[i, :] = cluster_mean(points_same_cluster)

    print("Cluster complete!")
    return centroid, cluster_assessment


In [12]:
feature_test = feature_use[:20]
centroids, clusterAssment = Kmeans(feature_test)

print(centroids)
print(clusterAssment)

Cluster complete!
[[ 5.43016667e+01  2.37500000e+00  5.25116667e+01  5.39600000e+01
   5.55183333e+01 -5.31166667e+00  8.09666667e+00  1.45833333e+00
   7.12000000e+00  7.77000000e+00  8.66500000e+00 -3.02000000e+00
   1.49166667e+00  2.00000000e-01  1.38000000e+00  1.45666667e+00
   1.56000000e+00 -3.56666667e-01  2.57166667e+00  2.66666667e-01
   2.37166667e+00  2.53833333e+00  2.77166667e+00 -6.51666667e-01
   1.33333333e-02  1.50000000e-02  1.66666667e-03  3.33333333e-03
   6.66666667e-03 -2.33333333e-02  7.00000000e-02  4.83333333e-02
   3.00000000e-02  5.66666667e-02  9.33333333e-02 -1.15000000e-01
   1.20346500e+00  8.53150000e-01 -6.06258333e-01  7.84731667e-01
   1.70436667e-01  1.75665000e-01]
 [ 6.21000000e+01  9.25000000e+00  5.39500000e+01  6.03850000e+01
   6.89000000e+01 -2.41400000e+01  1.63500000e+01  1.10050000e+01
   8.37500000e+00  1.22500000e+01  1.99700000e+01 -2.43100000e+01
   2.43500000e+00  1.15000000e+00  1.54500000e+00  2.03500000e+00
   3.00500000e+00 -2.97

In [13]:
# # 主成分分析
# def pca(data):
#     mean = cluster_mean(data)
#     # np.tile()默认复制x轴，如果有两个参数如(2,1)，则前者为复制y轴，后者为复制x轴



In [14]:
help(np.tile)

Help on function tile in module numpy:

tile(A, reps)
    Construct an array by repeating A the number of times given by reps.
    
    If `reps` has length ``d``, the result will have dimension of
    ``max(d, A.ndim)``.
    
    If ``A.ndim < d``, `A` is promoted to be d-dimensional by prepending new
    axes. So a shape (3,) array is promoted to (1, 3) for 2-D replication,
    or shape (1, 1, 3) for 3-D replication. If this is not the desired
    behavior, promote `A` to d-dimensions manually before calling this
    function.
    
    If ``A.ndim > d``, `reps` is promoted to `A`.ndim by pre-pending 1's to it.
    Thus for an `A` of shape (2, 3, 4, 5), a `reps` of (2, 2) is treated as
    (1, 1, 2, 2).
    
    Note : Although tile may be used for broadcasting, it is strongly
    recommended to use numpy's broadcasting operations and functions.
    
    Parameters
    ----------
    A : array_like
        The input array.
    reps : array_like
        The number of repetitions of `A`

In [15]:
type(np.zeros((3, 1)))

numpy.ndarray

In [16]:
a = np.zeros((2, 3))
print(type(a))
print(a)
b = np.mat(np.zeros((2, 3)))
print((type(b)))
print(b)


<class 'numpy.ndarray'>
[[0. 0. 0.]
 [0. 0. 0.]]
<class 'numpy.matrix'>
[[0. 0. 0.]
 [0. 0. 0.]]


In [17]:
a = np.ones((3, 2))
a[0, 0] = -1
a = -a
print(a)
a[:, 0] == 1

[[ 1. -1.]
 [-1. -1.]
 [-1. -1.]]


array([ True, False, False])

In [18]:
mean_points = cluster_mean(feature_use)
m = feature_use.shape[0]
points_distance_to_center = np.zeros(feature_use.shape[0])

for i in range(m):
    temp_dist = Euclidean_distance(feature_use[i], mean_points)
    points_distance_to_center[i] = temp_dist

points_distance_to_center = pd.DataFrame(points_distance_to_center)
name_list = feature[feature.columns[1]]
name_list = pd.DataFrame(name_list.values)
points_distance_to_center = pd.concat([name_list, points_distance_to_center], axis=1)

# points_distance_to_center.to_csv(r"./output/points_distance_to_center.csv")
points_distance_to_center.to_csv(r"./output/points_distance_to_center_42_features.csv")

In [19]:
points_distance_to_center = points_distance_to_center.values

In [20]:
points_distance_to_center

array([['1-L-1.wav', 15.403503209882548],
       ['1-L-2.wav', 14.76820145763773],
       ['2-L-1.wav', 17.14138939157319],
       ...,
       ['702-L-7.wav', 46.629848916881606],
       ['702-L-8.wav', 50.65519424497799],
       ['702-L-9.wav', 29.024704079893723]], dtype=object)

In [25]:
outlier_index = []
# np.zeros(61)
for i in range(len(points_distance_to_center)):
    if points_distance_to_center[i, 1] > 60:
        outlier_index.append(i)

np.matrix(outlier_index)


matrix([[  76,   77,  114,  115,  146,  147,  161,  162,  187,  205,
          206,  236,  237,  244,  245, 1096, 1098, 1099, 1100, 1175,
         1183, 1188, 1254, 1255, 1256, 1257, 1258, 1263, 1265, 1266,
         1267, 1268, 1269, 1270, 1274, 1342, 1343, 1346, 1348, 1349,
         1350, 1353, 2017, 3039, 4071, 4072, 4073, 4074, 4075, 4076,
         4077, 4078, 4079, 4080, 4081, 5216, 5217, 5221, 5222, 5223,
         5224, 5225, 5304, 5305, 5307, 5308, 5309, 5311, 5312, 5313]])

In [22]:
a = np.ones((3, 4))
a[0, 0] = 0
a[2, 2] = 4
print(a)
a = np.delete(a, 1, axis=0)
a

[[0. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 4. 1.]]


array([[0., 1., 1., 1.],
       [1., 1., 4., 1.]])

In [23]:
a = feature.columns[10:-1]
b = feature.columns[1]
a = np.insert(a, 0, b)
name = feature[b]
# name = np.delete(name,[1])
name.values

array(['1-L-1.wav', '1-L-2.wav', '2-L-1.wav', ..., '702-L-7.wav',
       '702-L-8.wav', '702-L-9.wav'], dtype=object)